<a href="https://colab.research.google.com/github/klordo/nlp_homeworks/blob/hw7/nlp_hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки

In [ ]:
!pip install transformers datasets

In [2]:
import json
from pathlib import Path
import numpy as np

import torch
from tqdm.notebook import tqdm
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertTokenizer
from transformers import DistilBertForQuestionAnswering
from transformers import pipeline

from torch.utils.data import DataLoader
from transformers import AdamW

In [3]:
from transformers import DPRReader, DPRReaderTokenizer

In [4]:
tqdm.pandas()

# Подготовка датасета

In [5]:
dataset = load_dataset("sberquad")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

Одна запись c плохим ответом, ее решил просто убрать

In [7]:
dataset['train'].filter(lambda x: x['id'] == 61603)[0]

{'id': 61603,
 'title': 'SberChallenge',
 'context': 'Четвёртый альбом Issues (1999) группы Korn также попал на вершину Billboard 200.[118][153] За один месяц он стал трижды платиновым.[154] За первую неделю он разошёлся тиражом 573,000 копий,[153] а первый сингл Falling Away from Me достиг 99 позиции в Billboard Hot 100.[155] Немногим раньше, до выхода альбома, Korn появляются в эпизоде Отличная загадка группы Korn о пиратском призраке мультсериала Южный парк, в котором состоялась премьера сингла Falling Away from Me .[156][157] В конце 1990-х и начале 2000-х различные ню-метал-группы, как, например, Korn,[158][159] Limp Bizkit[160][161] и P.O.D.,[162][163] стабильно участвуют в телепередаче Total Request Live на MTV.',
 'question': 'Сколько в первую неделю было продано копий альбома Issues группы Korn',
 'answers': {'text': ['Четвёртый альбом Issues (1999) группы Korn также попал на вершину Billboard 200.[118][153] За один месяц он стал трижды платиновым.[154] За первую неделю он раз

In [8]:
dataset['train'] = dataset['train'].filter(lambda x: x['id'] != 61603)

In [9]:
def get_dataset(dataset):
    dataset_dict = {'texts':[], 'answers':[], 'questions':[], 'starts':[], 'ends':[]}

    for item in dataset:
        text = item['context']
        answer = item['answers']['text'][0]
        start = item['answers']['answer_start'][0]
        end = start + len(answer)

        # иногда индекс начала сильно неправильный или равен -1
        # также в ответе бывают начала с большой буквы, не как в тексте
        # или в ответе не совпадают последние (первые) знаки препинания
        answer = answer.strip('.,?() ')
        start = text.lower().find(answer.lower())
        end = start + len(answer)
        answer = text[start:end]

        dataset_dict['texts'].append(text)
        dataset_dict['answers'].append(answer)
        dataset_dict['questions'].append(item['question'])
        dataset_dict['starts'].append(start)
        dataset_dict['ends'].append(end)

    return dataset_dict

Поздно вспомнил, что можно сразу было взять срез у датасета, но пришлось бы переписывать код

In [10]:
train_dataset = get_dataset(dataset['train'])
val_dataset = get_dataset(dataset['validation'])

In [11]:
TRAIN_LEN = 3000
VAL_LEN = 10

train_dataset['texts'] = train_dataset['texts'][:TRAIN_LEN]
train_dataset['answers'] = train_dataset['answers'][:TRAIN_LEN]
train_dataset['questions'] = train_dataset['questions'][:TRAIN_LEN]
train_dataset['starts'] = train_dataset['starts'][:TRAIN_LEN]
train_dataset['ends'] = train_dataset['ends'][:TRAIN_LEN]

val_dataset['texts'] = val_dataset['texts'][-VAL_LEN:]
val_dataset['answers'] = val_dataset['answers'][-VAL_LEN:]
val_dataset['questions'] = val_dataset['questions'][-VAL_LEN:]
val_dataset['starts'] = val_dataset['starts'][-VAL_LEN:]
val_dataset['ends'] = val_dataset['ends'][-VAL_LEN:]

In [12]:
train_dataset_dict = train_dataset
val_dataset_dict = val_dataset

# Дообучение модели

In [13]:
# model_name = 'kisa-misa/distilrubert-tiny-cased-conversational'
model_name = 'GeorgeKhlestov/distilbert_finetuned'
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

Считал сначала также, как мы раньше делали, но без treshold.

Длина была ~1800 и возникала ошибка при обучении в строке 'output = ...'

RuntimeError: The size of tensor a (608) must match the size of tensor b (512) at non-singleton dimension 1

Не смог понять в чем дело...

In [14]:
max_len = 400

In [15]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

tokenizer.model_max_length = max_len

In [16]:
train_encodings = tokenizer(train_dataset_dict['texts'],  train_dataset_dict['questions'], truncation=True, padding=True, max_length=max_len)
val_encodings = tokenizer(val_dataset_dict['texts'], val_dataset_dict['questions'], truncation=True, padding=True, max_length=max_len)

In [17]:
def add_token_positions(encodings, dataset_dict):

  start_positions = []
  end_positions = []
  for i in range(len(dataset_dict['starts'])):
      start_positions.append(encodings.char_to_token(i, dataset_dict['starts'][i]))
      end_positions.append(encodings.char_to_token(i, dataset_dict['ends'][i]))
      # if None, the answer passage has been truncated
      if start_positions[-1] is None:
          start_positions[-1] = tokenizer.model_max_length
      if end_positions[-1] is None:
          end_positions[-1] = tokenizer.model_max_length
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_dataset_dict)
add_token_positions(val_encodings, val_dataset_dict)

In [18]:
class SberquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SberquadDataset(train_encodings)
val_dataset = SberquadDataset(val_encodings)

In [19]:
NUM_EPOCHS = 7

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(NUM_EPOCHS):
    losses = []
    print("Epoch %s of %s" %(epoch + 1, NUM_EPOCHS))
    i = 0
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        losses.append(loss.item())
        loss.backward()
        optim.step()
        i += 1
    if i == 2: break
    print(np.array(losses).mean())
# model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 of 7
2.075613071309759
Epoch 2 of 7
0.8448113767390556
Epoch 3 of 7
0.5282733193103303
Epoch 4 of 7
0.33897083379486775
Epoch 5 of 7
0.26377144019971505
Epoch 6 of 7
0.17572759591201517
Epoch 7 of 7
0.13446874211126186


In [22]:
valid_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

for batch in valid_loader:
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

In [23]:
for i in range(len(val_dataset)):
    print('answer: ', val_dataset_dict['answers'][i])

    start = outputs['start_logits'][i].argmax()
    end = outputs['end_logits'][i].argmax()
    predict_answer_tokens = batch['input_ids'][i, start:end+1]
    print('predict:', tokenizer.decode(predict_answer_tokens), '\n')

answer:  653—661
predict: Джим Бойд, 

answer:  весной 1963
predict: 653 [UNK] 661 ) 

answer:  усадьбы Эусеби Гуэля
predict: Low, 

answer:  Джим Бойд
predict: смесь перхлората калия, антрацена и серы ) 

answer:  смесь бертолетовой соли, лактозы и канифоли
predict: в рубиновом лазере. 

answer:  в рубиновом лазере
predict: Lehman Brothers. 

answer:  Lehman Brothers
predict: Light Crust Doughboys, 

answer:  Low
predict: 1963 они были хорошо известны только в Ливерпуле, то в октябре того же года о них знала вся страна, 

answer:  Light Crust Doughboys
predict: в III веке до нашей эры. 

answer:  в III веке до нашей эры
predict: Эусеби Гуэля. 



Модель плохо предсказывает ответы. Видно, что некоторые ее предсказания заучены от других ответов, хотя выборки я не смешивал. Видимо в них присутствуют похожие вопросы и ответы.

Пути решения, которые я вижу:

Увеличить тренировочные данные

Возможно проблема в самом датасете, в моих изменениях в нем

Еще из-за max_len=400 могли тоже случиться проблемы